In [91]:
#!pip install -U pyLDAvis
#!pip install -U pandas

In [92]:
## imports
import os, sys
import pprint as pp
import unicodedata

In [93]:
## 一つ上の階層のファイルを見るように設定
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))

In [94]:
## term settings
term_classes        = [ 'spell', 'sound' ]
term_class          = term_classes[0]
ngram_is_inclusive  = True
## doc settings
max_doc_size        = 11
min_doc_size        =  3
print(f"max_doc_size: {max_doc_size}")
print(f"min_doc_size: {min_doc_size}")
### boundary handling
add_boundary       = False
boundary_mark      = "#"
## term setting
gap_mark           = "…"
term_is_skippy     = True
n_for_ngram        = 4
max_gap_ratio      = 0.8
max_gap_size       = round(max_doc_size * max_gap_ratio)
print(f"term_class: {term_class}")
print(f"term_is_skippy: {term_is_skippy}")
print(f"max_gap_size: {max_gap_size}")
print(f"n_for_ngram: {n_for_ngram}")
### accent handling
suppress_accents   = True
accent_marks       = [ "ˈ", "ˌ" ] 
if term_class == 'sound':
    if suppress_accents:
        accent_status = "-unaccented"
    else:
        accent_stratus = "-accented"
else:
    accent_status = ""
print(f"accent_status: {accent_status}")
## define term_type
if term_class == 'spell':
    if term_is_skippy:
        term_type = f"sp_skippy{n_for_ngram}gram"
    else:
        term_type = f"sp_{n_for_ngram}gram"
else:
    if term_is_skippy:
        term_type = f"sn_skippy{n_for_ngram}gram"
    else:
        term_type = f"sn_{n_for_ngram}gram"
## check
print(f"term_type: {term_type}")

max_doc_size: 11
min_doc_size: 3
term_class: spell
term_is_skippy: True
max_gap_size: 9
n_for_ngram: 4
accent_status: 
term_type: sp_skippy4gram


In [95]:
## target language
## a key must be part of a file name 
target_lang_dict = {    'en_US' : 'English (US)',
                        'en_UK' : 'English (UK)',
                        'en_N_only' : 'English noun (WN)',
                        'en_V_only' : 'English verb (WN)',
                        'en_A_only' : 'English adj (WN)',
                        'en_R_only' : 'English adv (WN)',
                        'ar'    : 'Arabic',
                        'de'    : 'German',
                        'de_N_only' : 'German Nouns',
                        'de_non_N_only' : 'German Non-nouns',
                        'eo'    : 'Esperanto',
                        'es_ES' : 'Spanish (Spain)',
                        'es_MX' : 'Spanish (Mexico)',
                        'fi'    : 'Finnish',
                        'fr_FR' : 'French (France)',
                        'fr_QC' : 'French (Quebec)',
                        'is'    : 'Icelandic',
                        'ir'    : 'Irish',
                        'nl'    : 'Dutch',
                        'ro'    : 'Romanian',
                        'sw'    : 'Swahili' }
## proper language selection
target_lang_keys = [    'en_US', # 0
                        'en_UK', # 1
                        'en_N_only', # 2
                        'en_V_only', # 3
                        'en_A_only', # 4
                        'en_R_only', # 5
                        'ar', # 6
                        'de', # 7
                        'de_N_only', # 8
                        'de_non_N_only', # 9
                        'eo', 'es_ES', 'es_MX',
                        'fi', 'fr_FR', 'fr_QC',
                        'is', 'nl', 'ro', 'sw',
                        'ir' # This lacks sound
                    ]
## check
target_lang_key  = target_lang_keys[4]
print(f"target_lang_key: {target_lang_key}")
print(f"target lang: {target_lang_dict[target_lang_key]} [{target_lang_key}]")
## target_attr [effective only for Irish]
target_class = ""
#target_class = None # This causes an unrediable error
if target_lang_key == "ir":
    target_classes = [ 'adjectives', 'nouns', 'verbs' ]
    target_class = f"-{target_classes[3]}"
print(f"target_class: {target_class}")

target_lang_key: en_A_only
target lang: English adj (WN) [en_A_only]
target_class: 


In [136]:
## LDA/HDP
apply_term_filtering = True
## The following parameters need to be relatively large to prevent "Row sum not equal 1" error
term_minfreq       = 2
## The following value is crucial to prevent "Row sum not equal 1" error
abuse_threshold    = 0.04 # larger value selects shorter units, smaller value selects longer units
min_bot_size       = 3
# number of terms listed for a given topic
n_terms_to_show    = 120

In [137]:
## sampling
source_sampling          = True
source_sampling_rate     = 0.5
source_sampling_max_size = 5000
second_sampling          = False
second_sampling_rate     = 0.7

In [138]:
## set target files
import glob
data_dir1     = "data/open-dict-ipa/data1"
data_dir2     = "data/open-dict-ipa/data1a"
data_dir3     = "data/wn3"
data_dir4     = "data/irish"
target_files  = glob.glob(f"{data_dir1}/*")
target_files2 = glob.glob(f"{data_dir2}/*")
target_files.extend(target_files2)
target_files3 = glob.glob(f"{data_dir3}/*")
target_files.extend(target_files3)
target_files4 = glob.glob(f"{data_dir4}/*")
target_files.extend(target_files4)
#
target_files = sorted([ file for file in target_files if ".csv" in file ])
pp.pprint(target_files)

['data/irish/word-irish-adjectives-spell.csv',
 'data/irish/word-irish-noun-phrases-spell.csv',
 'data/irish/word-irish-nouns-spell.csv',
 'data/irish/word-irish-possessives-spell.csv',
 'data/irish/word-irish-prepositions-spell.csv',
 'data/irish/word-irish-verbs-spell.csv',
 'data/open-dict-ipa/data1/ar.csv.gz',
 'data/open-dict-ipa/data1/de.csv.gz',
 'data/open-dict-ipa/data1/en_UK.csv.gz',
 'data/open-dict-ipa/data1/en_US.csv.gz',
 'data/open-dict-ipa/data1/eo.csv.gz',
 'data/open-dict-ipa/data1/es_ES.csv.gz',
 'data/open-dict-ipa/data1/es_MX.csv.gz',
 'data/open-dict-ipa/data1/fa.csv.gz',
 'data/open-dict-ipa/data1/fi.csv.gz',
 'data/open-dict-ipa/data1/fr_FR.csv.gz',
 'data/open-dict-ipa/data1/fr_QC.csv.gz',
 'data/open-dict-ipa/data1/is.csv.gz',
 'data/open-dict-ipa/data1/ja.csv.gz',
 'data/open-dict-ipa/data1/jam.csv.gz',
 'data/open-dict-ipa/data1/ma.csv.gz',
 'data/open-dict-ipa/data1/nb.csv.gz',
 'data/open-dict-ipa/data1/nl.csv.gz',
 'data/open-dict-ipa/data1/or.csv.gz',
 '

In [139]:
## get source data from files
import pandas as pd
import gzip
#target_language_key = "en_US" # can be changed to get other languages
#if target_class != "" or target_class is not None:
if target_class != "":
    target_file = [ f for f in target_files if target_lang_key in f and target_class in f ][0]
else:
    target_file = [ f for f in target_files if target_lang_key in f ][0]
print(f"processing: {target_file}")
##
if target_lang_key == "ir":
    col_names = ['spell', 'POS']
else:
    col_names = ['spell', 'sound']
#
if target_file.endswith(".gz"):
    with gzip.open(target_file, "rt") as f:
        raw_df = pd.read_csv(f, encoding = 'utf8', header = None, names = col_names )
else:
    with open(target_file, "rt") as f:
        raw_df = pd.read_csv(f, encoding = 'utf8', header = None, names = col_names )
## normalize characters
raw_df['spell'] = raw_df['spell'].apply(lambda x: unicodedata.normalize('NFC', str(x)))
## modify sound
try:
    sounds = raw_df['sound'].apply(lambda x: x.strip('/') )
    sounds = [ x.split("/,")[0] for x in sounds ] # picks up only the first of multiple entries
    raw_df['sound'] = sounds
except (AttributeError, KeyError):
    pass
#
raw_df.sample(10)

processing: data/wn3/en_A_only.csv


,spell,sound
3212,loose,ˈɫus
147,afoul,əˈfaʊɫ
3417,miscible,ˈmɪsəbəɫ
3675,numerical,nuˈmɛɹəkəɫ
935,cereal,ˈsɪɹiəɫ
4529,rural,ˈɹʊɹəɫ
1348,creditworthy,ˈkɹɛdɪtˌwɝði
2160,fifteen,ˈfɪfˈtin
4791,smoky,ˈsmoʊki
4683,shadowy,ˈʃædoʊi


In [140]:
## source sampling
len(raw_df)
if source_sampling:
	print(f"source sampling applied")
	if len(raw_df) >= source_sampling_max_size:
		raw_df = raw_df.sample(source_sampling_max_size)
	else:
		raw_df = raw_df.sample(round(len(raw_df) * source_sampling_rate))
## remove accent marking
if suppress_accents:
	try:
		raw_df['sound'] = raw_df['sound'].apply(lambda x: "".join([ y for y in list(x) if y not in accent_marks ]))
	except KeyError:
		pass
## add boudary marks
if add_boundary:
	raw_df['spell'] = raw_df['spell'].apply(lambda x: f"{boundary_mark}{x}{boundary_mark}")
	try:
		raw_df['sound'] = raw_df['sound'].apply(lambda x: f"{boundary_mark}{x}{boundary_mark}")
	except KeyError:
		pass
#
print(raw_df)

source sampling applied
              spell          sound
405     artiodactyl   ɑɹtioʊdæktəɫ
4500         rising         ɹaɪzɪŋ
2600           huge          hjudʒ
2634  hypercritical  haɪpɝkɹɪtɪkəɫ
4207         proved          pɹuvd
...             ...            ...
3471         morbid         mɔɹbəd
3903          pedal          pɛdəɫ
810          bugged           bəɡd
4296         raised          ɹeɪzd
5343    unappealing       ənəpiɫɪŋ

[5000 rows x 2 columns]


In [141]:
## generate 1-grams for spell and sound
## spell
raw_df['sp_1gram'] = raw_df['spell'].apply(lambda x: list(str(x)))
# add column of size
raw_df['sp_size'] = raw_df['sp_1gram'].apply(lambda x: len(x))
# add column of count of '-' inside
raw_df['hyphen'] = raw_df['sp_1gram'].apply(lambda x: list(x).count("-"))
# add column of count of '.' inside
raw_df['period'] = raw_df['sp_1gram'].apply(lambda x: list(x).count("."))
## sound
# takes the first entry, removes '/' around
try:
    raw_df['sn_1gram'] = raw_df['sound'].apply(lambda x: list(x) )
except (TypeError, KeyError):
    pass
# add column of size
try:
    raw_df['sn_size'] = raw_df['sn_1gram'].apply(lambda x: len(x))
except KeyError:
    pass
## check
raw_df

,spell,sound,sp_1gram,sp_size,hyphen,period,sn_1gram,sn_size
405,artiodactyl,ɑɹtioʊdæktəɫ,"[a, r, t, i, o, d, a, c, t, y, l]",11,0,0,"[ɑ, ɹ, t, i, o, ʊ, d, æ, k, t, ə, ɫ]",12
4500,rising,ɹaɪzɪŋ,"[r, i, s, i, n, g]",6,0,0,"[ɹ, a, ɪ, z, ɪ, ŋ]",6
2600,huge,hjudʒ,"[h, u, g, e]",4,0,0,"[h, j, u, d, ʒ]",5
2634,hypercritical,haɪpɝkɹɪtɪkəɫ,"[h, y, p, e, r, c, r, i, t, i, c, a, l]",13,0,0,"[h, a, ɪ, p, ɝ, k, ɹ, ɪ, t, ɪ, k, ə, ɫ]",13
4207,proved,pɹuvd,"[p, r, o, v, e, d]",6,0,0,"[p, ɹ, u, v, d]",5
...,...,...,...,...,...,...,...,...
3471,morbid,mɔɹbəd,"[m, o, r, b, i, d]",6,0,0,"[m, ɔ, ɹ, b, ə, d]",6
3903,pedal,pɛdəɫ,"[p, e, d, a, l]",5,0,0,"[p, ɛ, d, ə, ɫ]",5
810,bugged,bəɡd,"[b, u, g, g, e, d]",6,0,0,"[b, ə, ɡ, d]",4
4296,raised,ɹeɪzd,"[r, a, i, s, e, d]",6,0,0,"[ɹ, e, ɪ, z, d]",5


In [142]:
## filtering raw_data by size
print(f"term_type: {term_type}")
if "sp_" in term_type:
    df_filtered = raw_df[ (raw_df['sp_size'] <= max_doc_size) & (raw_df['sp_size'] >= min_doc_size) & (raw_df['hyphen'] == 0) & (raw_df['period'] == 0) ]
else:
    df_filtered = raw_df[ (raw_df['sn_size'] <= max_doc_size) & (raw_df['sn_size'] >= min_doc_size) ]
#
df_filtered

term_type: sp_skippy4gram


,spell,sound,sp_1gram,sp_size,hyphen,period,sn_1gram,sn_size
405,artiodactyl,ɑɹtioʊdæktəɫ,"[a, r, t, i, o, d, a, c, t, y, l]",11,0,0,"[ɑ, ɹ, t, i, o, ʊ, d, æ, k, t, ə, ɫ]",12
4500,rising,ɹaɪzɪŋ,"[r, i, s, i, n, g]",6,0,0,"[ɹ, a, ɪ, z, ɪ, ŋ]",6
2600,huge,hjudʒ,"[h, u, g, e]",4,0,0,"[h, j, u, d, ʒ]",5
4207,proved,pɹuvd,"[p, r, o, v, e, d]",6,0,0,"[p, ɹ, u, v, d]",5
410,ascertained,æsɝteɪnd,"[a, s, c, e, r, t, a, i, n, e, d]",11,0,0,"[æ, s, ɝ, t, e, ɪ, n, d]",8
...,...,...,...,...,...,...,...,...
3471,morbid,mɔɹbəd,"[m, o, r, b, i, d]",6,0,0,"[m, ɔ, ɹ, b, ə, d]",6
3903,pedal,pɛdəɫ,"[p, e, d, a, l]",5,0,0,"[p, ɛ, d, ə, ɫ]",5
810,bugged,bəɡd,"[b, u, g, g, e, d]",6,0,0,"[b, ə, ɡ, d]",4
4296,raised,ɹeɪzd,"[r, a, i, s, e, d]",6,0,0,"[ɹ, e, ɪ, z, d]",5


In [143]:
## define df after second sampling if any
len(df_filtered)
if second_sampling:
    df = df_filtered.sample(round(len(df_filtered) * second_sampling_rate))
else:
    df = df_filtered
len(df)

4450

In [144]:
## spell 2grams
#import ngrams
import gen_ngrams
module_name = "gen_ngrams"
reload_module = False
if reload_module:
    import importlib
    importlib.reload(module_name)

if term_class == 'spell':
    #sp_2grams = [ ngrams.list_gen_ngrams (x, n = 2, check = False) for x in df['sp_1gram'] ]
    sp_2grams = [ gen_ngrams.gen_ngrams (x, n = 2, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_2grams):
            g.extend(list(df['sp_1gram'])[i])
    ## add sp_2gram
    df['sp_2gram'] = sp_2grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_90425/3511996102.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_2gram'] = sp_2grams


In [145]:
## spell 3grams
#import ngrams
import gen_ngrams
if n_for_ngram > 2 and term_class == 'spell':
    #sp_3grams = [ ngrams.list_gen_ngrams (x, n = 3, check = False) for x in df['sp_1gram'] ]
    sp_3grams = [ gen_ngrams.gen_ngrams (x, n = 3, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_3grams):
            g.extend(list(df['sp_2gram'])[i])
    ## add sp_3gram
    df['sp_3gram'] = sp_3grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_90425/1405656247.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_3gram'] = sp_3grams


In [146]:
## spell 4grams
#import ngrams
import gen_ngrams
if n_for_ngram > 3 and term_class == 'spell':
    #sp_4grams = [ ngrams.list_gen_ngrams (x, n = 4, check = False) for x in df['sp_1gram'] ]
    sp_4grams = [ gen_ngrams.gen_ngrams (x, n = 4, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_4grams):
            g.extend(list(df['sp_3gram'])[i])
    ## add sp_4gram
    df['sp_4gram'] = sp_4grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_90425/1571879696.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_4gram'] = sp_4grams


In [147]:
## spell 5grams
#import ngrams
import gen_ngrams
if n_for_ngram > 4 and term_class == 'spell':
    sp_5grams = [ gen_ngrams.gen_ngrams (x, n = 5, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_5grams):
            g.extend(list(df['sp_4gram'])[i])
    ## add sp_5gram
    df['sp_5gram'] = sp_5grams

In [148]:
## spell skippy 2gram
#import ngrams_skippy
import gen_ngrams
reload_module = False
module_name = "gen_ngrams"
if reload_module:
    import importlib
    importlib.reload(module_name)
#
if term_class == 'spell':
    #sp_skippy2grams = [ ngrams_skippy.gen_skippy2grams(x, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sp_skippy2grams = [ gen_ngrams.gen_skippy_ngrams(x, 2, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_skippy2grams):
            g.extend(list(df['sp_1gram'])[i])
    #
    df['sp_skippy2gram'] = sp_skippy2grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_90425/2729548111.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_skippy2gram'] = sp_skippy2grams


In [149]:
## spell skippy 3gram
#import ngrams_skippy
import gen_ngrams
if n_for_ngram > 2 and term_class == 'spell':
    #sp_skippy3grams = [ ngrams_skippy.gen_skippy3grams(x, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sp_skippy3grams = [ gen_ngrams.gen_skippy_ngrams(x, 3, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_skippy3grams):
            g.extend(list(df['sp_skippy2gram'])[i])
    #
    df['sp_skippy3gram'] = sp_skippy3grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_90425/2630798607.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_skippy3gram'] = sp_skippy3grams


In [150]:
## spell skippy 4gram
#import ngrams_skippy
import gen_ngrams
if n_for_ngram > 3 and term_class == 'spell':
    #sp_skippy4grams = [ ngrams_skippy.gen_skippy4grams(x, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sp_skippy4grams = [ gen_ngrams.gen_skippy_ngrams(x, 4, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_skippy4grams):
            g.extend(list(df['sp_skippy3gram'])[i])
    #
    df['sp_skippy4gram'] = sp_skippy4grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_90425/1288956926.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_skippy4gram'] = sp_skippy4grams


In [151]:
## spell skippy 5gram
import gen_ngrams
if n_for_ngram > 4 and term_class == 'spell':
    sp_skippy5grams = [ gen_ngrams.gen_skippy_ngrams(x, 5, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_skippy5grams):
            g.extend(list(df['sp_skippy4gram'])[i])
    #
    df['sp_skippy5gram'] = sp_skippy5grams

In [152]:
## sound 2grams
#import ngrams
import gen_ngrams
module_name = "gen_ngrams"
reload_module = False
if reload_module:
    import importlib
    importlib.reload(module_name)
#
if term_class == 'sound':
    #sn_2grams = [ ngrams.list_gen_ngrams (x, n = 2, check = False) for x in df['sn_1gram'] ]
    sn_2grams = [ gen_ngrams.gen_ngrams(x, n = 2, sep ="", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_2grams):
            g.extend(list(df['sn_1gram'])[i])
    ## add sn_2gram
    df['sn_2gram'] = sn_2grams

In [153]:
## sound 3grams
#import ngrams
import gen_ngrams
if n_for_ngram > 2 and term_class == 'sound':
    #sn_3grams = [ ngrams.list_gen_ngrams (x, n = 3, check = False) for x in df['sn_1gram'] ]
    sn_3grams = [ gen_ngrams.gen_ngrams(x, n = 3, sep = "", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_3grams):
            g.extend(list(df['sn_2gram'])[i])
    ## add sn_3gram
    df['sn_3gram'] = sn_3grams

In [154]:
## sound 4grams
#import ngrams
import gen_ngrams
if n_for_ngram > 3 and term_class == 'sound':
    #sn_4grams = [ ngrams.list_gen_ngrams (x, n = 4, check = False) for x in df['sn_1gram'] ]
    sn_4grams = [ gen_ngrams.gen_ngrams (x, n = 4, sep = "", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_4grams):
            g.extend(list(df['sn_3gram'])[i])
    ## add sn_4gram
    df['sn_4gram'] = sn_4grams

In [155]:
## sound 5grams
import gen_ngrams
if n_for_ngram > 4 and term_class == 'sound':
    sn_5grams = [ gen_ngrams.gen_ngrams (x, n = 5, sep = "", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_5grams):
            g.extend(list(df['sn_4gram'])[i])
    ## add sn_4gram
    df['sn_5gram'] = sn_5grams

In [156]:
## sound skippy 2gram
#import ngrams_skippy
import gen_ngrams
if term_class == 'sound':
    #sn_skippy2grams = [ ngrams_skippy.gen_skippy2grams(x, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sn_skippy2grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 2, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy2grams):
            g.extend(list(df['sn_1gram'])[i])
    #
    df['sn_skippy2gram'] = sn_skippy2grams

In [157]:
## sound skippy 3gram
#import ngrams_skippy
import gen_ngrams
if n_for_ngram > 2 and term_class == 'sound':
    #sn_skippy3grams = [ ngrams_skippy.gen_skippy3grams(x, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sn_skippy3grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 3, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy3grams):
            g.extend(list(df['sn_skippy2gram'])[i])
    #
    df['sn_skippy3gram'] = sn_skippy3grams

In [158]:
## sound skippy 4gram
#import ngrams_skippy
import gen_ngrams
if n_for_ngram > 3 and term_class == 'sound':
    #sn_skippy4grams = [ ngrams_skippy.gen_skippy4grams(x, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sn_skippy4grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 4, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy4grams):
            g.extend(list(df['sn_skippy3gram'])[i])
    #
    df['sn_skippy4gram'] = sn_skippy4grams

In [159]:
## sound skippy 5gram
import gen_ngrams
if n_for_ngram > 4 and term_class == 'sound':
    sn_skippy5grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 5, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy5grams):
            g.extend(list(df['sn_skippy4gram'])[i])
    #
    df['sn_skippy5gram'] = sn_skippy5grams

In [160]:
## check df
dropped_vars = [ 'sp_size', 'hyphen', 'period', 'sn_size' ]
if term_class == 'spell':
    extra = [ 'sn_1gram', 'sn_2gram', 'sn_3gram', 'sn_4gram',
             'sn_skippy2gram', 'sn_skippy3gram', 'sn_skippy4gram' ]
    dropped_vars.extend(extra)
    target_vars = [ x for x in df.columns if not x in dropped_vars ]
else:
    extra = [ 'sp_1gram', 'sp_2gram', 'sp_3gram', 'sp_4gram',
             'sp_skippy2gram', 'sp_skippy3gram', 'sp_skippy4gram' ]
    dropped_vars.extend(extra)
    target_vars = [ x for x in df.columns if not x in dropped_vars ]    
#
df[target_vars]

,spell,sound,sp_1gram,sp_2gram,sp_3gram,sp_4gram,sp_skippy2gram,sp_skippy3gram,sp_skippy4gram
405,artiodactyl,ɑɹtioʊdæktəɫ,"[a, r, t, i, o, d, a, c, t, y, l]","[ar, rt, ti, io, od, da, ac, ct, ty, yl, a, r,...","[art, rti, tio, iod, oda, dac, act, cty, tyl, ...","[arti, rtio, tiod, ioda, odac, dact, acty, cty...","[ar, a…t, a…i, a…o, a…d, a…a, a…c, a…t, rt, r…...","[art, ar…i, ar…o, ar…d, ar…a, ar…c, ar…t, a…ti...","[arti, art…o, art…d, art…a, art…c, art…t, ar…i..."
4500,rising,ɹaɪzɪŋ,"[r, i, s, i, n, g]","[ri, is, si, in, ng, r, i, s, i, n, g]","[ris, isi, sin, ing, ri, is, si, in, ng, r, i,...","[risi, isin, sing, ris, isi, sin, ing, ri, is,...","[ri, r…s, r…i, r…n, r…g, is, i…i, i…n, i…g, si...","[ris, ri…i, ri…n, ri…g, r…si, r…s…n, r…s…g, r…...","[risi, ris…n, ris…g, ri…in, ri…i…g, ri…ng, r…s..."
2600,huge,hjudʒ,"[h, u, g, e]","[hu, ug, ge, h, u, g, e]","[hug, uge, hu, ug, ge, h, u, g, e]","[huge, hug, uge, hu, ug, ge, h, u, g, e]","[hu, h…g, h…e, ug, u…e, ge, ug, u…e, ge, ge, h...","[hug, hu…e, h…ge, uge, uge, hu, h…g, h…e, ug, ...","[huge, hug, hu…e, h…ge, uge, uge, hu, h…g, h…e..."
4207,proved,pɹuvd,"[p, r, o, v, e, d]","[pr, ro, ov, ve, ed, p, r, o, v, e, d]","[pro, rov, ove, ved, pr, ro, ov, ve, ed, p, r,...","[prov, rove, oved, pro, rov, ove, ved, pr, ro,...","[pr, p…o, p…v, p…e, p…d, ro, r…v, r…e, r…d, ov...","[pro, pr…v, pr…e, pr…d, p…ov, p…o…e, p…o…d, p…...","[prov, pro…e, pro…d, pr…ve, pr…v…d, pr…ed, p…o..."
410,ascertained,æsɝteɪnd,"[a, s, c, e, r, t, a, i, n, e, d]","[as, sc, ce, er, rt, ta, ai, in, ne, ed, a, s,...","[asc, sce, cer, ert, rta, tai, ain, ine, ned, ...","[asce, scer, cert, erta, rtai, tain, aine, ine...","[as, a…c, a…e, a…r, a…t, a…a, a…i, a…n, sc, s…...","[asc, as…e, as…r, as…t, as…a, as…i, as…n, a…ce...","[asce, asc…r, asc…t, asc…a, asc…i, asc…n, as…e..."
...,...,...,...,...,...,...,...,...,...
3471,morbid,mɔɹbəd,"[m, o, r, b, i, d]","[mo, or, rb, bi, id, m, o, r, b, i, d]","[mor, orb, rbi, bid, mo, or, rb, bi, id, m, o,...","[morb, orbi, rbid, mor, orb, rbi, bid, mo, or,...","[mo, m…r, m…b, m…i, m…d, or, o…b, o…i, o…d, rb...","[mor, mo…b, mo…i, mo…d, m…rb, m…r…i, m…r…d, m…...","[morb, mor…i, mor…d, mo…bi, mo…b…d, mo…id, m…r..."
3903,pedal,pɛdəɫ,"[p, e, d, a, l]","[pe, ed, da, al, p, e, d, a, l]","[ped, eda, dal, pe, ed, da, al, p, e, d, a, l]","[peda, edal, ped, eda, dal, pe, ed, da, al, p,...","[pe, p…d, p…a, p…l, ed, e…a, e…l, da, d…l, al,...","[ped, pe…a, pe…l, p…da, p…d…l, p…al, eda, ed…l...","[peda, ped…l, pe…al, p…dal, edal, edal, ped, p..."
810,bugged,bəɡd,"[b, u, g, g, e, d]","[bu, ug, gg, ge, ed, b, u, g, g, e, d]","[bug, ugg, gge, ged, bu, ug, gg, ge, ed, b, u,...","[bugg, ugge, gged, bug, ugg, gge, ged, bu, ug,...","[bu, b…g, b…g, b…e, b…d, ug, u…g, u…e, u…d, gg...","[bug, bu…g, bu…e, bu…d, b…gg, b…g…e, b…g…d, b…...","[bugg, bug…e, bug…d, bu…ge, bu…g…d, bu…ed, b…g..."
4296,raised,ɹeɪzd,"[r, a, i, s, e, d]","[ra, ai, is, se, ed, r, a, i, s, e, d]","[rai, ais, ise, sed, ra, ai, is, se, ed, r, a,...","[rais, aise, ised, rai, ais, ise, sed, ra, ai,...","[ra, r…i, r…s, r…e, r…d, ai, a…s, a…e, a…d, is...","[rai, ra…s, ra…e, ra…d, r…is, r…i…e, r…i…d, r…...","[rais, rai…e, rai…d, ra…se, ra…s…d, ra…ed, r…i..."


In [161]:
## select data type and define doc_dict
import random
if "sp_" in term_type:
    base_type = "spell"
else:
    base_type = "sound"
doc_dict = { i: x for i, x in enumerate(df[base_type]) }
## check
random.sample(doc_dict.items(), 10)

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_90425/1181347170.py:9: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(doc_dict.items(), 10)


[(1417, 'salient'),
 (2296, 'apathetic'),
 (182, 'rentable'),
 (2440, 'farsighted'),
 (2013, 'marked'),
 (2698, 'flaky'),
 (1033, 'descending'),
 (2545, 'rightish'),
 (1745, 'homesick'),
 (208, 'much')]

In [162]:
## select bots for analysis
enable_term_change = False # if you want to change term_type to save time and energy
if enable_term_change:
	term_type = 'sp_skippy4gram'
print(f"(changed) term_type: {term_type}")

## bot stands for 'bag-of-terms', a generalization of 'bag-of-words'
bots = [ x for x in df[term_type] if len(x) > min_bot_size ] # Crucially
import random
random.sample(bots, 1)

(changed) term_type: sp_skippy4gram


[['simi',
  'sim…l',
  'sim…a',
  'sim…r',
  'si…il',
  'si…i…a',
  'si…i…r',
  'si…la',
  'si…l…r',
  'si…ar',
  's…mil',
  's…mi…a',
  's…mi…r',
  's…m…la',
  's…m…l…r',
  's…m…ar',
  's…ila',
  's…il…r',
  's…i…ar',
  's…lar',
  'imil',
  'imi…a',
  'imi…r',
  'im…la',
  'im…l…r',
  'im…ar',
  'i…ila',
  'i…il…r',
  'i…i…ar',
  'i…lar',
  'mila',
  'mil…r',
  'mi…ar',
  'm…lar',
  'ilar',
  'imil',
  'imi…a',
  'imi…r',
  'im…la',
  'im…l…r',
  'im…ar',
  'i…ila',
  'i…il…r',
  'i…i…ar',
  'i…lar',
  'mila',
  'mil…r',
  'mi…ar',
  'm…lar',
  'ilar',
  'mila',
  'mil…r',
  'mi…ar',
  'm…lar',
  'ilar',
  'ilar',
  'sim',
  'si…i',
  'si…l',
  'si…a',
  'si…r',
  's…mi',
  's…m…l',
  's…m…a',
  's…m…r',
  's…il',
  's…i…a',
  's…i…r',
  's…la',
  's…l…r',
  's…ar',
  'imi',
  'im…l',
  'im…a',
  'im…r',
  'i…il',
  'i…i…a',
  'i…i…r',
  'i…la',
  'i…l…r',
  'i…ar',
  'mil',
  'mi…a',
  'mi…r',
  'm…la',
  'm…l…r',
  'm…ar',
  'ila',
  'il…r',
  'i…ar',
  'lar',
  'imi',
  'im…l',
  '

In [163]:
## generate dictionary
from gensim.corpora import Dictionary
diction = Dictionary(bots)
print(diction)
#apply_term_filtering = False
if apply_term_filtering:
    print(f"term filtering applied")
    diction.filter_extremes(no_below = term_minfreq, no_above = abuse_threshold)
else:
    print(f"term filtering not applied")
## check
print(diction)
## generate DTM
corpus = [ diction.doc2bow(bot) for bot in bots if len(bot) > min_bot_size ] # Crucially

Dictionary<186772 unique tokens: ['a', 'ac', 'act', 'acty', 'act…l']...>
term filtering applied
Dictionary<89612 unique tokens: ['ac', 'act', 'act…l', 'ac…l', 'ac…y']...>


In [164]:
## HDP (n_topics = 15)
import gensim.models
import pyLDAvis.gensim
max_n_topics = 15
hdp15 = gensim.models.HdpModel(corpus, diction, T = max_n_topics, random_state = 1)
vis_data15 = pyLDAvis.gensim.prepare(hdp15, corpus, diction)
pyLDAvis.display(vis_data15)

In [165]:
## save LDAvis output as a html file
save_LDAvis = True
if save_LDAvis:
	lang_dir_name = target_lang_dict[target_lang_key].split()[0]
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data15, vis_output)

In [166]:
## topic investigation
import numpy as np
import HDP_helper
reload_module = False
if reload_module:
    import importlib
    importlib.reload(HDP_helper)

documents_topics = np.zeros([hdp15.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in hdp15[c]:
        documents_topics[topic_id][doc_id] = prob

n_docs_to_show  = 10
n_terms_to_pick = 60
hdp15.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    topic_t = hdp15.print_topic(topic_id, topn = n_terms_to_pick)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_pick)}")
    print(f"nonzero count: ", len(probs.nonzero()[0]))
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.001 * ry + 0.001 * ul + 0.001 * et + 0.001 * m…c + 0.001 * abl + 0.001 * e…en + 0.001 * e…nt + 0.001 * e…e…t + 0.001 * able + 0.001 * n…al + 0.0 * me + 0.0 * g…a + 0.0 * i…nt + 0.0 * i…ic + 0.0 * g…t + 0.0 * n…nt + 0.0 * t…ic + 0.0 * i…e…t + 0.0 * cal + 0.0 * ra…e + 0.0 * ant + 0.0 * mi + 0.0 * ge + 0.0 * ial + 0.0 * r…nt + 0.0 * el + 0.0 * i…en + 0.0 * id + 0.0 * o…y + 0.0 * ica + 0.0 * e…ent + 0.0 * at…c + 0.0 * om + 0.0 * ic…l + 0.0 * g…n + 0.0 * t…y + 0.0 * g…l + 0.0 * ce + 0.0 * r…ic + 0.0 * ate + 0.0 * s…ic + 0.0 * th + 0.0 * t…al + 0.0 * e…y + 0.0 * to + 0.0 * e…ti + 0.0 * er…e + 0.0 * e…t…l + 0.0 * en…a + 0.0 * l…nt + 0.0 * n…en + 0.0 * lar + 0.0 * nt…l + 0.0 * s…nt + 0.0 * sti + 0.0 * si + 0.0 * d…nt + 0.0 * m…ic + 0.0 * tr + 0.0 * g…d
nonzero count:  1774
	0.9991: indefinable
	0.9991: preparatory
	0.9991: indifferent
	0.9991: sedimentary
	0.9991: unorganized
	0.9991: rectangular
	0.9991: ventricular
	0.9991: reorganized
	0.9991: irreparable
	0.9991: documentary


In [167]:
## save topic structures
#hdp.get_topics() # =/= show_topics()
#hdp.print_topics()
hdp_topics = hdp15.show_topics(num_topics = max_n_topics,
                               num_words = n_terms_to_show, formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }
## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
#hdp15_topics_out = f"results/terms-by-topics-raw/hdp{max_n_topics}_topics_raw.csv"
hdp15_topics_out = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics{max_n_topics}-{term_type}{accent_status}.csv"
topics_df.to_csv(hdp15_topics_out, header = False, index = None)

In [168]:
## HDP (n_topics = 45)
import gensim.models
import pyLDAvis.gensim
max_n_topics = 45
hdp45 = gensim.models.HdpModel(corpus, diction, T = max_n_topics, random_state = 1)
vis_data45 = pyLDAvis.gensim.prepare(hdp45, corpus, diction)
pyLDAvis.display(vis_data45)

In [169]:
## save LDAvis output as a html file
lang_dir_name = target_lang_dict[target_lang_key].split()[0]
save_LDAvis = True
if save_LDAvis:
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data45, vis_output)

In [170]:
## save topic structures
#hdp.get_topics() # =/= show_topics()
#hdp.print_topics()
hdp_topics = hdp45.show_topics(num_topics = max_n_topics,
                               num_words = n_terms_to_show, formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }
## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
#hdp45_topics_out = f"results/terms-by-topics-raw/hdp{max_n_topics}_topics_raw.csv"
hdp45_topics_out = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics{max_n_topics}-{term_type}{accent_status}.csv"
topics_df.to_csv(hdp45_topics_out, header = False, index = None)

In [171]:
## topic investigation
import numpy as np
import HDP_helper
documents_topics = np.zeros([hdp45.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in hdp45[c]:
        documents_topics[topic_id][doc_id] = prob
#
n_docs_to_show  = 10
n_terms_to_pick = 10
hdp45.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    topic_t = hdp45.print_topic(topic_id, topn = n_terms_to_pick)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_pick)}")
    print(f"nonzero count: {len(probs.nonzero()[0])}")
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.002 * ive + 0.002 * t…v + 0.002 * t…ve + 0.002 * tiv + 0.002 * tive + 0.001 * ate + 0.001 * ct + 0.001 * e…ti + 0.001 * cal + 0.001 * e…v
nonzero count: 798
	0.9990: destructive
	0.9990: instructive
	0.9990: unnecessary
	0.9990: restrictive
	0.9990: unobtrusive
	0.9990: insensitive
	0.9990: preparatory
	0.9990: prospective
	0.9990: competitive
	0.9990: comparative
topic_id 1: 0.001 * ss + 0.001 * m…c + 0.001 * ess + 0.001 * et + 0.001 * l…ss + 0.001 * les + 0.001 * le…s + 0.001 * less + 0.001 * om + 0.001 * me
nonzero count: 635
	0.9990: unconfirmed
	0.9990: unconvinced
	0.9990: indifferent
	0.9989: consonantal
	0.9989: incontinent
	0.9989: independent
	0.9989: continental
	0.9989: transparent
	0.9989: econometric
	0.9989: residential
topic_id 2: 0.002 * ate + 0.001 * at…d + 0.001 * ated + 0.001 * i…at + 0.001 * i…a…e + 0.001 * ul + 0.001 * i…ate + 0.001 * l…nt + 0.001 * g…a + 0.001 * g…d
nonzero count: 571
	0.9990: particulate
	0.9990: amalgamated
	0.9990: stimulating
	0

In [172]:
## HDP (n_topics = 90)
import gensim.models
import pyLDAvis.gensim
max_n_topics = 90
hdp90 = gensim.models.HdpModel(corpus, diction, T = max_n_topics,
                               random_state = 1,
                               #var_converge = 0.001
                               )
vis_data90 = pyLDAvis.gensim.prepare(hdp90, corpus, diction)
pyLDAvis.display(vis_data90)

In [173]:
## save LDAvis output as a html file
save_LDAvis = True
if save_LDAvis:
	lang_dir_name = target_lang_dict[target_lang_key].split()[0]
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data90, vis_output)

In [174]:
## save topic structures
hdp_topics = hdp90.show_topics(num_topics = max_n_topics,
                               num_words = n_terms_to_show, formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }
## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
#hdp90_topics_out = f"results/terms-by-topics-raw/hdp{max_n_topics}_topics_raw.csv"
hdp90_topics_out = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics{max_n_topics}-{term_type}{accent_status}.csv"
topics_df.to_csv(hdp90_topics_out, header = False, index = None)

In [175]:
## topic investigation
import numpy as np
import HDP_helper
documents_topics = np.zeros([hdp90.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in hdp90[c]:
        documents_topics[topic_id][doc_id] = prob
## investigate topics
n_docs_to_show  = 10
n_terms_to_pick = 10
hdp90.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    #topic_encoding = ", ".join(hdp.show_topic(topic_id))
    topic_t = hdp90.print_topic(topic_id, topn = n_terms_to_pick)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_pick)}")
    print(f"nonzero count: ", len(probs.nonzero()[0]))
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.004 * ive + 0.003 * t…v + 0.003 * t…ve + 0.003 * tiv + 0.002 * tive + 0.002 * s…v + 0.002 * s…ve + 0.002 * e…v + 0.002 * ct + 0.002 * ate
nonzero count:  523
	0.9990: destructive
	0.9990: instructive
	0.9990: incompetent
	0.9990: inoffensive
	0.9990: restrictive
	0.9990: unobtrusive
	0.9990: insensitive
	0.9990: prospective
	0.9990: progressive
	0.9990: competitive
topic_id 1: 0.001 * et + 0.001 * ce + 0.001 * d…nt + 0.001 * li…e + 0.001 * i…nt + 0.001 * i…e…t + 0.001 * e…en + 0.001 * e…nt + 0.001 * n…nt + 0.001 * e…ti
nonzero count:  454
	0.9989: incontinent
	0.9989: independent
	0.9989: continental
	0.9989: residential
	0.9989: generalized
	0.9989: kinesthetic
	0.9989: pessimistic
	0.9989: sympathetic
	0.9989: nonresident
	0.9989: obsolescent
topic_id 2: 0.001 * nic + 0.001 * me + 0.001 * s…ic + 0.001 * f…n + 0.001 * om + 0.001 * i…ic + 0.001 * a…t…c + 0.001 * ot + 0.001 * on…c + 0.001 * e…h
nonzero count:  369
	0.9989: indifferent
	0.9989: unorganized
	0.9989: reorgani